## 100bit Analysis
### Drawing in a Community with Limitations
#### Crawling 100bit
---
This is the jupyter notebook for the functions that deal with crawling 100bit's webpages then storing the data in a .xlsx file. If you would like to see the functions that graph the data, see "100 Bit Project."

The code below defines the functions used for crawling and scraping data from 100bit. I was originally planning on using BeautifulSoup for crawling 100bit, as I had used it for a previous project for an AI class. However, BeautifulSoup is incapable of grabbing javascript variables, which is how 100bit stores its data. So, Selenium came in handy for simulating a chrome user retrieving 100bit's javascript variables through chrome's debug console.

100bit's data is obfuscated as variables with short letter names, but with a bit of experimentation (and help from friends of the community), I was able to figure out the main variables that control 100bit's grid. It's possible to play around with these variables yourself, e.g., type "u" into chrome's dev console to see the order of user uploads to a particular No..

In [17]:
# Using Selenium to run a browser's javascript code with javaa referenced from: https://stackoverflow.com/questions/63965653/how-do-i-run-command-from-web-browser-console-in-python
# and also: https://stackoverflow.com/questions/64717302/deprecationwarning-executable-path-has-been-deprecated-selenium-python
# and also also: https://stackoverflow.com/questions/48666620/python-selenium-webdriver-stuck-at-get-in-a-loop
# And: https://stackoverflow.com/questions/16180428/can-selenium-webdriver-open-browser-windows-silently-in-the-background

from asyncio.windows_events import NULL
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import xlsxwriter
import pandas as pd
import sys


#The maximum characters that can be written in one cell of a .xlsx file
MAX_CELL_SIZE = 32767


def findLatestNo():
    '''
    findLatestNo()
    A function to determine what the latest finished No. is.
    Returns a number correlating to the latest No. (e.g., 1 for No. 1).
    The latest No. should be the first No. that can be seen in the top left of: https://dan-ball.jp/en/javagame/bit/history/
    '''
    currentNo = 585 #Start at No. 585 as there are at least 585 No.s at the time of writing
    NoExists = True
    print("Checking for new No.s starting at ", currentNo ,"...")
    while(NoExists):
        #URL = 'https://dan-ball.jp/en/javagame/bit/' + str(currentNo) + '.html'
        URL = 'https://dan-ball.jp/en/javagame/bit/?code=' + str(currentNo)
        ChromeOptions = webdriver.ChromeOptions()
        ChromeOptions.add_argument('--disable-browser-side-navigation')
        ChromeOptions.add_argument("--headless")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=ChromeOptions)
        driver.get(URL)
        sleep(3)
        placements = driver.execute_script("return p")
        #if not all(item is None for item in placements):
        if not (placements[0] is None):
            print("\tNo.", currentNo, "exists.")
            currentNo += 1
        else:
            NoExists = False
            currentNo -= 1
    print("\tThe latest No. in the history is: No.", currentNo)
    return currentNo

def toGrid(grid):
    '''
    toGrid()
    A print function used for debugging.
    grid = an array with 256 * 192 elements with 1's and 0's to represent black/white pixels respectively
    prints ASCII art representing the grid.
    '''
    output = ""
    for i, x in enumerate(grid):
        if ((i % 256) == 0):
            output += "\n"
        if (x == 1):
            #output += u"▯"
            output += "1"
        else:
            #output += u"▮"
            output += "0"
    return output

def listToString(list, typeOfData = "default"):
    '''
    listToString()
    A function to convert various crawled data into string lists for storing into the 100bit dump.
    Certain values are ignored so that I do not have to filter them out later (there are a lot of extra empty
    values in array elements, for instance).
    list = The javascript list variable that was crawled
    typeOfData = a string indicating what kind of data the list correlates to.
    Returns a comma-delimited string version of the list
    '''
    output = ""
    first = True
    previousItem = 0
    for x in list:
        # Some of the data types have leftover junk data, so I use switch case to lop it off.
        if typeOfData == "users":
            if (x == ""):
                return output
        elif typeOfData == "dotPlacements":
            if (x is None):
                return output
        elif typeOfData == "dotOwner":
            if ((x == 0) and previousItem != 0):
                return output 
        if first:
            output += str(x)
            first = False
        else:
            output += "," + str(x)
        previousItem = x
    return output

def doWBWrite(wb, row, col, data):
    '''
    doWBWrite()
    A function I wrote to assist in writing to the workbook, as I frequently need to write more data that
    can be fit in 1 cell of the .xlsx file. If the data to write is bigger than MAX_CELL_SIZE, remaining
    data will be written in the same column on the next row, and the row after that and so on until all data is written.
    wb = the workbook object to write to
    row = which row to start writing to
    col = which column to start writing to
    data = the data to be written, either an int or a string usually.
    '''
    if (type(data) == int):
        wb.write(row, col, data)
        return
    dataSize = len(data)
    nextRow = row
    dataWritten = 0
    while (dataSize > dataWritten):
        wb.write(nextRow, col, data[dataWritten:(dataWritten+MAX_CELL_SIZE)])
        nextRow += 1
        dataWritten += MAX_CELL_SIZE

def doCrawl(OUTPUT_FILE, STARTING_NO, ENDING_NO):
    '''
    doCrawl()
    The main engine for crawling. If the OUTPUT_FILE already exists, the crawler engine will check to determine which
    No.s are missing, and fill them in accordingly.
    OUTPUT_FILE = Which .xlsx file this crawl should write to. The .xlsx file may or may not already exist.
    STARTING_NO = The No. to start crawling from.
    ENDING_NO = The No. to stop crawling on (inclusive).
    '''
    print("Crawling from", STARTING_NO, "to", ENDING_NO,"and outputting to", OUTPUT_FILE)
    # Open the URL you want to execute JS
    currentNo = STARTING_NO
    try:
        # Writing to .xlsx files referenced from: https://stackoverflow.com/questions/25883017/xlsxwriter-and-xlwt-writing-a-list-of-strings-to-a-cell
        workbook = xlsxwriter.Workbook(OUTPUT_FILE)
        # write multiple rows
        while (currentNo <= ENDING_NO):
            print("Scanning: No. " + str(currentNo) + " (out of " + str(ENDING_NO) + ")")
            worksheet = workbook.add_worksheet("No." + str(currentNo))
            #URL = 'https://dan-ball.jp/en/javagame/bit/' + str(currentNo) + '.html'
            URL = 'https://dan-ball.jp/en/javagame/bit/?code=' + str(currentNo)
            #driver = webdriver.Chrome(chromedriver)
            ChromeOptions = webdriver.ChromeOptions()
            ChromeOptions.add_argument('--disable-browser-side-navigation')
            ChromeOptions.add_argument("--headless")
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=ChromeOptions)
            driver.get(URL)
            sleep(3)
            NoObject = {}
            NoObject["NoNumber"] = currentNo
            driver.execute_script("x = 30000")
            NoObject["NoDots"] = driver.execute_script("return x")
            NoObject["users"] = driver.execute_script("return u")
            NoObject["startingGrid"] = driver.execute_script("return g")
            NoObject["dotPlacements"] = driver.execute_script("return p")
            NoObject["dotOwner"] = driver.execute_script("return aa")
            if (NoObject["dotPlacements"][0] is None):
                #Occasionally, 100bit's webpages will load .html without loading javascript variables.
                #This is a failsafe to prevent that.
                print("Failed to get data from No.", str(currentNo), ". Retrying!")
                driver.quit()
                continue
            # Add header, referenced from: https://stackoverflow.com/questions/55797151/formatting-only-headers-with-data-using-xlsxwriter
            for col, data in enumerate(["No", "Dot Count", "Users", "Starting Grid", "Dot Placements", "Dot Owner"]):
                worksheet.write(0, col, data)
            doWBWrite(wb=worksheet, row=1, col=0, data=NoObject["NoNumber"])
            doWBWrite(wb=worksheet, row=1, col=1, data=NoObject["NoDots"])
            doWBWrite(wb=worksheet, row=1, col=2, data=listToString(NoObject["users"], "users"))
            doWBWrite(wb=worksheet, row=1, col=3, data=listToString(NoObject["startingGrid"], "startingGrid"))
            doWBWrite(wb=worksheet, row=1, col=4, data=listToString(NoObject["dotPlacements"], "dotPlacements"))
            doWBWrite(wb=worksheet, row=1, col=5, data=listToString(NoObject["dotOwner"], "dotOwner"))
            currentNo += 1
            driver.quit()
    finally:
        driver.quit()
        workbook.close()
        print("Done writing data!")


def main(args):
    OUTPUT_FILE = "100bitdump.xlsx"
    startNo = 1
    endNo = -1
    #wb = pd.read_excel(INPUT_FILE, sheet_name=None, header=None, names=["No", "Dot Count", "Users", "Starting Grid", "Dot Placements", "Dot Owner"])
    if len(args) >= 1:
        OUTPUT_FILE = args[0]
    if len(args) >= 2:
        startNo = int(args[1])
        endNo = int(args[1])
    if len(args) >= 3:
        endNo = int(args[2])
    doCrawl(OUTPUT_FILE, startNo, endNo)


Over time, new No.'s are added to 100bit after enough edits to the present No. are made. At the time of writing, the latest No. is at No. 586, however, I made a function to determine what the current completed No. is for crawling the newer No.'s

In [18]:
latestNo = findLatestNo()
print("Latest No. is currently:", latestNo)



====== WebDriver manager ======


Checking for new No.s starting at  585 ...


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
<ipython-input-17-cb8f6152fc4b>:36: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=ChromeOptions)


====== WebDriver manager ======


	No. 585 exists.


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


====== WebDriver manager ======


...

	No. 597 exists.


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


====== WebDriver manager ======


	No. 598 exists.


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


	The latest No. in the history is: No. 598
Latest No. is currently: 598


The driver function to start the crawler. Originally, I ran the code on a command line, hence why the function is called "main". By default, the function below will create the file "testdump.xlsx" from No. 580 to whatever the latest No. is. Be warned: the .xlsx writer will overwrite existing .xlsx files, and I haven't added a way for the code to modify and "fill in" missing No.'s on an existing .xlsx file

In [19]:
main(["testdump.xlsx", 580, latestNo])



====== WebDriver manager ======


Crawling from 580 to 598 and outputting to testdump.xlsx
Scanning: No. 580 (out of 598)


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
<ipython-input-17-cb8f6152fc4b>:148: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=ChromeOptions)


====== WebDriver manager ======


Scanning: No. 581 (out of 598)


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


====== WebDriver manager ======


...

Scanning: No. 597 (out of 598)


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


====== WebDriver manager ======


Scanning: No. 598 (out of 598)


Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\artis\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


Done writing data!


If there were no issues with the internet, then the resulting .xlsx file should be usable by the "100 Bit Project.ipynb" notebook.

Crawling the entire 100bit database originally took about an hour and a half, but fortunately running the code for creating the heatmaps takes much, much less time.

There was some trial and error with the crawling process. Namely, I found out the hard way that if you write more characters to a single cell in a .xlsx file than it can handle, any extra data gets left off with no warnings. As a result, I was missing about half of the data for edits, and most other data wasn't formatted correctly. I later discovered this fact when I noticed that applying edits to the start of No. X did NOT result in a grid that was the same as the start of No. (X + 1).

Before that, I also did some tests with different ways to store the database. Storing a lot of data in a .txt file proved to scale poorly, as text files struggle to open when they are several Megabytes large. Likewise, .csv files proved to be a poor means of storing data due to their size limits.

While I think I should have used a different library for writing .xlsx files (such as one that allows re-editing existing .xlsx files), the one I chose did the trick well enough for crawling and writing 100bit's entire history in one swoop.